In [1]:
# !git clone https://github.com/Megvii-BaseDetection/YOLOX

In [2]:
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm
import shutil
def modify_xml_jpg_file(data_dir, out_dir):
    jpg_path = list()
    xml_path = list()
    file_names = set()
    
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".jpg"):
                if 'checkpoint' in file:
                    pass
                else:
                    file_path = os.path.join(root, file)
                    file_names.add(os.path.splitext(file_path.split(os.sep)[-1])[0])
                    jpg_path.append(file_path)
    
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith(".xml"):
                if 'checkpoint' in file:
                    pass
                else:
                    file_path = os.path.join(root, file)
                    file_names.add(os.path.splitext(file_path.split(os.sep)[-1])[0])
                    xml_path.append(file_path)
    
    file_names = sorted(list(file_names))
    name_map = dict((n, i) for i, n in enumerate(file_names))
    
    for xml_file in tqdm(xml_path):
        xmlTree = ET.parse(xml_file)
        xml_name = os.path.splitext(xml_file.split(os.sep)[-1])[0]
        int_name = name_map[xml_name]
        rootElement = xmlTree.getroot()
        rootElement.findall("path")[0].text = f'{int_name}.jpg'
        rootElement.findall("filename")[0].text = f'{int_name}.jpg'

        sub_path = os.path.join(*xml_file.split(os.sep)[1:-1])
        write_dir = os.path.join(out_dir, sub_path)
        if os.path.exists(write_dir):
            pass
        else:
            os.makedirs(write_dir)

        write_path = os.path.join(write_dir, str(int_name)+'.xml')
        xmlTree.write(write_path, encoding='UTF-8', xml_declaration=True)
            
    for jpg_file in tqdm(jpg_path):
        jpg_name = os.path.splitext(jpg_file.split(os.sep)[-1])[0]
        int_name = name_map[jpg_name]

        sub_path = os.path.join(*jpg_file.split(os.sep)[1:-1])
        write_dir = os.path.join(out_dir, sub_path)
        if os.path.exists(write_dir):
            pass
        else:
            os.makedirs(write_dir)
        write_path = os.path.join(write_dir, str(int_name)+'.jpg')
        shutil.copy2(jpg_file, write_path)         


In [3]:
modify_xml_jpg_file('linmao-new', out_dir='linmao-new_processed')


100%|██████████| 8407/8407 [04:59<00:00, 28.07it/s] 


# (Train/Validation split data)

In [4]:
import os

dataset_directory = 'linmao-new_processed'

train_directory = './train'
validation_directory = './validation'

os.makedirs(train_directory, exist_ok=True)
os.makedirs(validation_directory, exist_ok=True)

In [5]:
import glob
import shutil
import random

train_ratio = 0.8

annotation_list = sorted(glob.glob(dataset_directory +'/Annotations'+ '/*.xml'))
image_list = sorted(glob.glob(dataset_directory +'/JPEGImages' + '/*.jpg'))

file_num = len(annotation_list)

index_list = list(range(file_num - 1))
random.shuffle(index_list)

for count, index in enumerate(tqdm(index_list)):
    if count < int(file_num * train_ratio):
        shutil.copy2(annotation_list[index], train_directory)
        shutil.copy2(image_list[index], train_directory)
    else:
        shutil.copy2(annotation_list[index], validation_directory)
        shutil.copy2(image_list[index], validation_directory)

100%|██████████| 8406/8406 [03:14<00:00, 43.23it/s] 


# (Convert Pascal VOC format to MS COCO format)

In [6]:
# !git clone https://github.com/Kazuhito00/voc2coco.git

In [7]:
# put this into voc2coco.py

label_map = {'nl_0438': 0,
             'nl_0431': 1,
             'nl_0239': 2,
             'nl_0238': 3,
             'nl_0271': 4,
             'nl_0280': 5,
             'nl_0433': 6,
             'nl_0224': 7,
             'nl_0098': 8}

In [8]:
!python voc2coco.py train train/train_annotations.json
!python voc2coco.py validation validation/validation_annotations.json

Number of xml files: 6725
Success: train/train_annotations.json
Number of xml files: 1681
Success: validation/validation_annotations.json


# (Training data directory preparation)

In [9]:
!mkdir dataset
!mkdir dataset/images
!mkdir dataset/images/train2017
!mkdir dataset/images/val2017
!mkdir dataset/images/annotations

!cp -rf train/*.jpg dataset/images/train2017
!cp -rf validation/*.jpg dataset/images/val2017
!cp -rf train/train_annotations.json dataset/images/annotations
!cp -rf validation/validation_annotations.json dataset/images/annotations

# Train

In [10]:
!rm -rf YOLOX/dataset

In [11]:
!mv dataset YOLOX/dataset

In [12]:
%cd YOLOX/
# !wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_nano.pthz

/home/jupyter/YOLOX_Run/YOLOX


In [13]:
# Copy train.py to root, since it needs to import from yolox
# !cp tools/train.py ./

In [ ]:
# Modify nano.py for epochs and other info

In [ ]:
!python train.py -f nano.py -d 1 -b 16 --fp16 -o -c yolox_nano.pth

2022-01-23 23:57:07 | INFO     | yolox.core.trainer:126 - args: Namespace(batch_size=16, cache=False, ckpt='yolox_nano.pth', devices=1, dist_backend='nccl', dist_url=None, exp_file='nano.py', experiment_name='nano', fp16=True, machine_rank=0, name=None, num_machines=1, occupy=True, opts=[], resume=False, start_epoch=None)
2022-01-23 23:57:07 | INFO     | yolox.core.trainer:127 - exp value:
╒══════════════════╤═══════════════════════════════╕
│ keys             │ values                        │
╞══════════════════╪═══════════════════════════════╡
│ seed             │ None                          │
├──────────────────┼───────────────────────────────┤
│ output_dir       │ './YOLOX_outputs'             │
├──────────────────┼───────────────────────────────┤
│ print_interval   │ 10                            │
├──────────────────┼───────────────────────────────┤
│ eval_interval    │ 1                             │
├──────────────────┼───────────────────────────────┤
│ num_classes      │ 9  

# (Inference test)

In [ ]:
# !cp tools/demo.py ./

## modify demo.py get class names
## names can be set up at YOLOX/yolox/data/datasets/limao_classes.py

In [ ]:
TEST_IMAGE_PATH = "dataset/images/val2017/"
MODEL_PATH = "YOLOX_outputs/nano/best_ckpt.pth"

!python demo.py image \
    -f nano.py \
    -c {MODEL_PATH} \
    --path {TEST_IMAGE_PATH} \
    --conf 0.25 \
    --nms 0.45 \
    --tsize 640 \
    --save_result \
    --device gpu

In [ ]:
# from PIL import Image

# OUTPUT_IMAGE_PATH = "/content/YOLOX/YOLOX_outputs/nano/vis_res/2021_09_29_17_46_56/000050.jpg" 
# Image.open(OUTPUT_IMAGE_PATH)

# ONNX出力(Export ONNX Model)

In [ ]:
!python tools/export_onnx.py \
    --output-name yolox_nano.onnx \
    -n yolox-nano \
    -f nano.py \
    -c {MODEL_PATH}

In [ ]:
!python demo/ONNXRuntime/onnx_inference.py \
    -m yolox_nano.onnx \
    -i {TEST_IMAGE_PATH} \
    -o ./ \
    -s 0.3 \
    --input_shape 416,416

In [ ]:
from PIL import Image

OUTPUT_IMAGE_PATH = "000050.jpg" 
Image.open(OUTPUT_IMAGE_PATH)

# ONNX -> TensorFlow 変換

In [ ]:
!pip install onnx-tf

In [ ]:
!onnx-tf convert \
    -i yolox_nano.onnx \
    -o yolox_nano_pb

# TensorFlow -> TensorFlow-Lite 変換

In [ ]:
!pip install tf-nightly

In [ ]:
import tensorflow as tf

In [ ]:
%cd /content/YOLOX

In [ ]:
# ダイナミックレンジ量子化
converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open('yolox_nano_dynamic_range_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 半精度浮動小数点数の量子化
converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quantized_model = converter.convert()

open('yolox_nano_float16_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 完全整数量子化
import glob
import numpy as np
from PIL import Image

test_image_pathlist = glob.glob('/content/YOLOX-Colaboratory-Training-Sample/01.image/*.jpg')
test_image_pathlist = test_image_pathlist[:100]
print(len(test_image_pathlist))

def representative_dataset():
    for test_image_path in test_image_pathlist:
        image = np.array(Image.open(test_image_path))
        image = image.astype('float32')
        image = tf.image.resize(image, (416, 416))
        image = image - 127.5
        image = image * 0.007843
        image = tf.transpose(image, perm=[2, 0, 1])
        image = tf.reshape(image, [1, 3, 416, 416])
        yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_quantized_model = converter.convert()

open('yolox_nano_int8_quantize.tflite', 'wb').write(tflite_quantized_model)

In [ ]:
# 完全整数量子化(入力含む)
import glob
import numpy as np
from PIL import Image

test_image_pathlist = glob.glob('/content/YOLOX-Colaboratory-Training-Sample/01.image/*.jpg')
test_image_pathlist = test_image_pathlist[:100]
print(len(test_image_pathlist))

def representative_dataset():
    for test_image_path in test_image_pathlist:
        image = np.array(Image.open(test_image_path))
        image = image.astype('float32')
        image = tf.image.resize(image, (416, 416))
        image = image - 127.5
        image = image * 0.007843
        image = tf.transpose(image, perm=[2, 0, 1])
        image = tf.reshape(image, [1, 3, 416, 416])
        yield [image]

converter = tf.lite.TFLiteConverter.from_saved_model('yolox_nano_pb')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quantized_model = converter.convert()

open('yolox_nano_only_int8_quantize.tflite', 'wb').write(tflite_quantized_model)